## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [1]:
import glob

import numpy as np
import scipy as sp
import scipy.io
from scipy.signal import filtfilt, butter
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

fs = 125


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def BandpassFilter(signal, pass_band=(40/60,240/60), fs=125):
    """
    Butterworth bandpass filter algorithm.
    
    Returns:
        Bandpass filtered signal
    """          
    b, a = sp.signal.butter(2, pass_band, btype='bandpass', fs = fs)
    return sp.signal.filtfilt(b, a, signal)

def FourierTransform(signal, fs):
    """
    Run a Fourier Transform on a signal
    
    Returns:
        Freq and Magnitude of the signal
    """
    freqs = np.fft.rfftfreq(2 * len(signal), 1 / fs)
    fft = np.abs(np.fft.rfft(signal, 2 * len(signal)))
    return freqs, fft

def createFeatures(ppg, accmag, fs):
    """
    Creates features for Random Forest Regressor.
    
    Returns:
        Dominant frequency for PPG & Accelerometer Magnitude signals
    """
    
    freqs, ppg_fft = FourierTransform(ppg, fs)
    freqs, accmag_fft = FourierTransform(accmag, fs)
    
    ppg_dominant_freq = freqs[np.argmax(ppg_fft)]
    accmag_dominant_freq = freqs[np.argmax(accmag_fft)]
    
    return np.array([ppg_dominant_freq, accmag_dominant_freq])

def randomForestRegression(features, labels):
    """
    Trains Random Forest Regression model.
    
    Returns:
        Trained model
    """
    
    clf = RandomForestRegressor(n_estimators=250, max_depth=8, random_state=43)
    
    kf = KFold(n_splits=5)
    
    for train_idx, test_idx in kf.split(features, labels):
        train_X, train_y = features[train_idx], labels[train_idx]
        clf.fit(train_X, train_y)
    
    return clf
    
def RunPulseRateAlgorithm(data_fl, ref_fl):
    
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ref_bpm  = sp.io.loadmat(ref_fl)['BPM0'].squeeze()
    fs = 125
    
    # Caculates magnitude of accelerometer
    acc_mag = np.sqrt(np.sum(np.square(np.vstack((accx, accy, accz))), axis = 0))
    
    # The dataset gives the BPM value in every 8-second time window and two successive time windows overlap by 6 seconds.
    window_length_fs = 8 * fs
    window_shift_fs = 2 * fs
    
    features, filt_signals = [], []
    errors, confidence = [], []
    labels = []
    k = 0
    
    for i in range(0, len(ppg) - window_length_fs, window_shift_fs):
        
        # Bandpass signal between 40-240BPM.
        filt_ppg = BandpassFilter(ppg[i : i + window_shift_fs])
        filt_accmag = BandpassFilter(acc_mag[i : i + window_shift_fs])
        
        filt_signals.append([filt_ppg, filt_accmag])
        
        # Compute features for random forest model.
        features.append(createFeatures(filt_ppg, filt_accmag, fs))
        labels.append(ref_bpm[k])
        k += 1
        
    features = np.array(features)
    
    # Trains RandomForestRegressor model which will estimate pulse rate (in BPM).
    model = randomForestRegression(features, np.array(labels))
    
    for i in range(len(filt_signals)):
        filt_ppg, filt_accmag = filt_signals[i]
        feature, true_bpm = features[i], labels[i]
        
        # Predicts the estimated pulse rate using trained random forest model.
        est_bpm = model.predict(feature.reshape((1, -1)))[0]
        
        # Calculates absolute error b/w ground-truth BPM and estimated BPM.
        errors.append(np.abs(true_bpm - est_bpm))
        
        freqs, ppg_fft = FourierTransform(ppg, fs)
        
        window_f = 45 / 60
        est_frequency_window = (freqs > (est_bpm / 60) - window_f) & (freqs < (est_bpm / 60) + window_f)
        
        # Confidence is estimated by summing the frequency spectrum near the pulse rate estimate and dividing it by the sum of the entire spectrum.
        ppg_power = np.sum(ppg_fft[est_frequency_window])
        total_power = np.sum(ppg_fft)
        confidence.append(ppg_power / total_power)
        
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    return np.array(errors), np.array(confidence)
Evaluate()

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


8.607935876849949